## Train a model locally or remote

In [20]:
%cd ~/SageMaker/ezsm-ray-FM/dev/

/home/ec2-user/SageMaker/ezsm-ray-FM/dev


In [4]:
%pip uninstall -y ezsmdeploy --quiet
%pip install -e ../ --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip uninstall -y ezsmdeploy --quiet
# !pip install --upgrade pip
%pip install ezsmdeploy==1.93.dev0

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


### Note: you may need to restart the kernel to use updated packages.

## Testing serverless huggingface models

In [11]:
# %pip uninstall -y sagemaker
# %pip install --upgrade sagemaker==2.148.0 --no-cache-dir

In [12]:
import sagemaker
sagemaker.__version__

'2.171.0'

In [13]:
!pip show ezsmdeploy

Name: ezsmdeploy
Version: 1.93.dev0
Summary: SageMaker custom deployments made easy
Home-page: https://pypi.python.org/pypi/ezsmdeploy
Author: Shreyas Subramanian
Author-email: subshrey@amazon.com
License: MIT
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages
Requires: boto3, sagemaker, sagemaker-studio-image-build, shortuuid, yaspin
Required-by: 


In [46]:
import ezsmdeploy

In [47]:
ezonsm = ezsmdeploy.Deploy(model = "dynamic-tinybert",
                           huggingface_model=True,
                           huggingface_model_task='question-answering',
                           serverless=True, 
                           serverless_memory=6144
                           )

0:00:00.079969 | created model(s). Now deploying on Serverless!
0:02:32.510625 | deployed model
0:02:32.510677 | Done! ✔ 


In [42]:
# Example for most models
payload = {"inputs":"The symbol *** marks the separator between the \
# prescribed beginning and the student’s completion:\nOnce upon a time, in an ancient house, there lived a girl named Lily. She loved"} 
           # # example for distilbert-base-uncased-finetuned-sst-2-english
# payload = {
#   "inputs": "the mesmerizing performances of the leads keep the film grounded and keep the audience riveted .",
# }

# # Example for distilbert / dynamic-tinybert / deepset/tinyroberta-squad2
# payload  = {"inputs": {
#     "question": "Who discovered silk?",
#     "context": "Legend has it that the process for making silk cloth was first invented by the wife of the Yellow Emperor, Leizu, around the year 2696 BC. The idea for silk first came to Leizu while she was having tea in the imperial gardens." + "The production of silk originates in China in the Neolithic (Yangshao culture, 4th millennium BCE). Silk remained confined to China until the Silk Road opened at some point during the later half of the first millennium BCE. "
# }}

# # Example for chat
# payload = {"inputs":"<human>: Who is Tesla?\n<bot>:",
#            "parameters":{"max_new_tokens":100}}


response = ezonsm.predictor.predict(payload)

print(response)

{'score': 0.3589574098587036, 'start': 107, 'end': 112, 'answer': 'Leizu'}


In [48]:
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)


In [51]:
from sagemaker.serializers import DataSerializer

ezonsm.predictor.serializer = DataSerializer(content_type='image/x-image')

In [54]:
ezonsm.predictor.predict(image.tobytes())

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "\nDetrForObjectDetection requires the timm library but it was not found in your environment. You can install it with pip:\n`pip install timm`. Please note that you may need to restart your runtime after installation.\n"
}
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/ezsm-hf-endpoint-hffafmzw2advbq3dvhrwy5 in account 607789152815 for more information.

In [37]:
ezonsm.predictor.predict("retro serie of different cars with different colors and shapes, mdjrny-v4 style")

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "\u0027str\u0027 object has no attribute \u0027pop\u0027"
}
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/ezsm-hf-endpoint-uvbgb4mqoublzsd75wq3qt in account 607789152815 for more information.

In [45]:
ezonsm.predictor.delete_endpoint()